In [1]:
! ~/.local/bin/aws s3 ls dev-rhassan/tony/

2020-04-11 14:11:22      62233 tmax001.png
2020-04-11 14:11:30      61915 tmax002.png


In [2]:
! ~/.local/bin/aws s3 ls dev-rhassan/tony/ | wc

      2       8      86


In [3]:
! ~/.local/bin/aws s3 rm s3://dev-rhassan/tony/ --recursive

delete: s3://dev-rhassan/tony/tmax001.png
delete: s3://dev-rhassan/tony/tmax002.png


In [4]:
! ~/.local/bin/aws s3 ls dev-rhassan/tony/

In [5]:
! ~/.local/bin/aws s3 ls dev-rhassan/

                           PRE TRR_test_cases/
                           PRE africa-scene-li/
                           PRE africa-scene-lists/
                           PRE newsslcert/
2020-04-10 15:00:03        315 get-cw-log-stream
2020-04-10 13:15:07     276255 lpgs_integration_testing_test_case_512_TC-512-10000-list.txt
2020-04-01 19:06:46      89930 tc_512_cloud_end_to_end.pdf


In [6]:
from playLib.bo_bucket import bo_get_bucket_list
bucket_name = 'ga-et-data'
prefix='Cloud_Veg_ET/Data/TMAX/'
lista=bo_get_bucket_list(bucket_name, prefix)

List count = 366


In [33]:
lista[0:5]

['Cloud_Veg_ET/Data/TMAX/tmax_001_gw.tif.npy',
 'Cloud_Veg_ET/Data/TMAX/tmax_002_gw.tif.npy',
 'Cloud_Veg_ET/Data/TMAX/tmax_003_gw.tif.npy',
 'Cloud_Veg_ET/Data/TMAX/tmax_004_gw.tif.npy',
 'Cloud_Veg_ET/Data/TMAX/tmax_005_gw.tif.npy']

In [7]:

# This is serial and VERY SLOW!
import os

bucket='ga-et-data'
for i in range(2):
    
    file_obj=lista[i]
    prefix = '/'.join(file_obj.split('/')[0:-1]) + '/'
    #print(prefix)
    filen = file_obj.split('/')[-1]
    #print(filen)
    argv = ','.join([bucket,prefix,filen])
    print(argv)
    cmd = 'python3 ./png-maker/png-maker.py ' + argv
    os.system(cmd) 

ga-et-data,Cloud_Veg_ET/Data/TMAX/,tmax_001_gw.tif.npy
ga-et-data,Cloud_Veg_ET/Data/TMAX/,tmax_002_gw.tif.npy


In [8]:
! ~/.local/bin/aws s3 ls dev-rhassan/tony/ | wc

      2       8      86


In [9]:
! ~/.local/bin/aws s3 ls dev-rhassan/tony/

2020-04-11 14:12:49      62233 tmax001.png
2020-04-11 14:12:58      61915 tmax002.png


# Part 2 Queues with RabitMQ

In [10]:
 ! sudo systemctl status rabbitmq-server.service | grep Active

   Active: inactive (dead) since Sat 2020-04-11 01:19:25 UTC; 12h ago


In [11]:
# hello

In [12]:
%%bash
#systemctl --type=service
systemctl status rabbitmq-server.service |grep Active


   Active: inactive (dead) since Sat 2020-04-11 01:19:25 UTC; 12h ago


In [13]:
! (cd rabbit; docker-compose up -d)

Creating network "rabbit_default" with the default driver
Creating rabbit_web_1 ... 
Creating rabbit_redis_1 ... 
Creating rabbit_tony_1 ... 
Creating rabbitmq ... 
Creating rabbit_qbarf_1 ... 
Creating rabbit_redis_1
Creating rabbit_web_1
Creating rabbit_tony_1
Creating rabbitmq
Creating rabbit_qbarf_1
ting rabbitmq ... done0m

In [14]:
! echo sleeping...

sleeping...


In [15]:

%%bash
for i in {1..10}; do
    echo waiting... ;
    sleep 1 ;
done

waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...


In [16]:
from playLib.qbird import Qbird

q = Qbird('localhost','tonyq')
q.q_declare()
msg='COVID-19 bites!!!!'
q.q_publish(msg)

In [17]:
for i in range(10):
    msg = 'COVID-19 bites!!!!' +str(i)
    q.q_publish(msg)

In [18]:
! docker ps

CONTAINER ID        IMAGE                          COMMAND                  CREATED             STATUS              PORTS                                                                                        NAMES
116edf47cdf9        tbutzer/q_barfer               "/usr/bin/python3 q_…"   17 seconds ago      Up 12 seconds                                                                                                    rabbit_qbarf_1
533112463ee7        tbutzer/bash-tester            "docker-entrypoint.s…"   17 seconds ago      Up 13 seconds       6379/tcp                                                                                     rabbit_tony_1
0e3c66575b83        rabbitmq:3-management-alpine   "docker-entrypoint.s…"   17 seconds ago      Up 10 seconds       4369/tcp, 5671/tcp, 0.0.0.0:5672->5672/tcp, 15671/tcp, 25672/tcp, 0.0.0.0:15672->15672/tcp   rabbitmq
4964386a5ad6        redis:alpine                   "docker-entrypoint.s…"   17 seconds ago      Up 14 seconds       6379

In [19]:
! (cd rabbit; docker-compose logs |grep qbarf | tail -20)

Attaching to rabbit_qbarf_1, rabbit_tony_1, rabbitmq, rabbit_redis_1, rabbit_web_1
qbarf_1     | 2020-04-11 14:13:03,940 - Q_BARFER - INFO - Logging Begins
qbarf_1     | 2020-04-11 14:13:08,998 - Q_BARFER - INFO - sleeping - waiting ... rabbitmq where are you?
qbarf_1     | 2020-04-11 14:13:12,002 - Q_BARFER - INFO - sleeping - waiting ... rabbitmq where are you?
qbarf_1     | 2020-04-11 14:13:15,008 - Q_BARFER - INFO - sleeping - waiting ... rabbitmq where are you?
qbarf_1     | 2020-04-11 14:13:16,306 - Q_BARFER - INFO -  [x] Received b'COVID-19 bites!!!!'
qbarf_1     | 2020-04-11 14:13:16,315 - Q_BARFER - INFO -  [x] Received b'COVID-19 bites!!!!0'
qbarf_1     | 2020-04-11 14:13:16,317 - Q_BARFER - INFO -  [x] Received b'COVID-19 bites!!!!1'
qbarf_1     | 2020-04-11 14:13:16,319 - Q_BARFER - INFO -  [x] Received b'COVID-19 bites!!!!2'
qbarf_1     | 2020-04-11 14:13:16,320 - Q_BARFER - INFO -  [x] Received b'COVID-19 bites!!!!3'
qbarf_1     | 2020-04-11 14:13:16,323 - Q_BARFER - INFO

In [20]:
! sudo docker exec rabbitmq rabbitmqctl status

Status of node rabbit@0e3c66575b83 ...
Runtime

OS PID: 331
OS: Linux
Uptime (seconds): 10
RabbitMQ version: 3.8.3
Node name: rabbit@0e3c66575b83
Erlang configuration: Erlang/OTP 22 [erts-10.7.1] [source] [64-bit] [smp:2:2] [ds:2:2:10] [async-threads:64]
Erlang processes: 448 used, 1048576 limit
Scheduler run queue: 1
Cluster heartbeat timeout (net_ticktime): 60

Plugins

Enabled plugin file: /etc/rabbitmq/enabled_plugins
Enabled plugins:

 * rabbitmq_management
 * rabbitmq_web_dispatch
 * rabbitmq_management_agent
 * amqp_client
 * cowboy
 * cowlib

Data directory

Node data directory: /var/lib/rabbitmq/mnesia/rabbit@0e3c66575b83

Config files

 * /etc/rabbitmq/rabbitmq.conf

Log file(s)

 * <stdout>

Alarms

(none)

Memory

Calculation strategy: rss
Memory high watermark setting: 0.4 of available memory, computed to: 3.3124 gb
code: 0.0302 gb (29.16 %)
other_proc: 0.0263 gb (25.39 %)
other_system: 0.024 gb (23.19 %)
allocated_unused: 0.0144 gb (13.9 %)
other_ets: 0.0029 gb (2.79 %)
r

In [21]:
! sudo docker exec rabbitmq rabbitmqctl list_queues

Timeout: 60.0 seconds ...
Listing queues for vhost / ...
name	messages
tonyq	0


# Now Consume those Right here

In [22]:
def callback(ch, method, properties, body):
    print(" [x] Received %r" % body)

In [23]:
qr=Qbird('localhost','tonyq')
qr.q_set_callback(callback)


# I used to consume the queue items here

## But now i have the qbarf service to do that in docker-compose and docker swarm!

In [34]:
# Dont blok here anymore
# qr.q_consume()

In [25]:
! sudo docker exec rabbitmq rabbitmqctl list_queues

Timeout: 60.0 seconds ...
Listing queues for vhost / ...
name	messages
tonyq	0


# Q them again

In [26]:
q = Qbird('localhost','tonyq')
q.q_declare()
for i in range(10):
    msg = 'COVID-19 STILL bites!!!!' +str(i)
    q.q_publish(msg)

In [27]:
! sudo docker exec rabbitmq rabbitmqctl list_queues

Timeout: 60.0 seconds ...
Listing queues for vhost / ...
name	messages
tonyq	0


# Drain these messages using the admin gui port 

http://localhost:15672/
        
## ssh tunnel via 03<tab> 15672 mhonk

In [29]:
! sudo docker exec rabbitmq rabbitmqctl list_queues

Timeout: 60.0 seconds ...
Listing queues for vhost / ...
name	messages
tonyq	0


In [30]:
! sudo netstat -anlp |grep docker-proxy |grep LISTEN

tcp6       0      0 :::8000                 :::*                    LISTEN      25016/docker-proxy  
tcp6       0      0 :::5672                 :::*                    LISTEN      25101/docker-proxy  
tcp6       0      0 :::15672                :::*                    LISTEN      25049/docker-proxy  


In [31]:
! curl localhost:8000
    
! curl localhost:8000

! curl localhost:8000

Hello World! I have been seen 1 times.
Hello World! I have been seen 2 times.
Hello World! I have been seen 3 times.


## Shut down the services with docker-compose

In [32]:
# SHUT Er DOWN NOW!!!!!!!
! (cd rabbit; docker-compose down)

Stopping rabbit_qbarf_1 ... 
Stopping rabbit_tony_1  ... 
Stopping rabbitmq       ... 
Stopping rabbit_redis_1 ... 
Stopping rabbit_web_1   ... 
ping rabbitmq       ... doneRemoving rabbit_qbarf_1 ... 
Removing rabbit_tony_1  ... 
Removing rabbitmq       ... 
Removing rabbit_redis_1 ... 
Removing rabbit_web_1   ... 
Removing network rabbit_defaultne
